In [2]:
from cot import Collection
collection = Collection("all")

Reusing dataset aqua_dataset (C:\Users\ottsi\.cache\huggingface\datasets\aqua_dataset\thoughtsource\1.0.0\3eba5363ba1758e1a49898a999cf1b3154283fdd1fc5303f34cecf1c30e4a728)


Loading aqua...


  0%|          | 0/3 [00:00<?, ?it/s]

Reusing dataset asdiv_dataset (C:\Users\ottsi\.cache\huggingface\datasets\asdiv_dataset\thoughtsource\1.0.0\966005651a36c08a795e8dd02bde9cb87c799ab0153ca91b6b1fd646341714db)


Loading asdiv...


...

Reusing dataset svamp_dataset (C:\Users\ottsi\.cache\huggingface\datasets\svamp_dataset\thoughtsource\1.0.0\1748ab9f83b8e03d7ff2c135169d0b2e35b1f62b0ae0195f02b60717d7ca0789)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading worldtree...


Reusing dataset worldtree_dataset (C:\Users\ottsi\.cache\huggingface\datasets\worldtree_dataset\thoughtsource\1.0.0\9a721c0ece6622af4eff7a43d81272a737c8e3c7f356268cef5a4bae0450e90d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [31]:
collection["aqua"]["train"][1]

{'id': '1',
 'ref_id': '',
 'question': 'In the coordinate plane, points (x, 1) and (5, y) are on line k. If line k passes through the origin and has slope 1/5, then what are the values of x and y respectively?',
 'type': 'multiplechoice',
 'choices': ['4 and 1', '1 and 5', '5 and 1', '3 and 5', '5 and 3'],
 'context': '',
 'cot': ['Line k passes through the origin and has slope 1/5 means that its equation is y=1/5*x.',
  'Thus: (x, 1)=(5, 1) and (5, y) = (5,1) -->x=5 and y=1',
  'Answer: C'],
 'answer': ['5 and 1'],
 'generated_cot': [],
 'feedback': []}

In [29]:
dataset = "worldtree"
for split in collection[dataset]:
    print(split)
    c = 0
    for x in collection[dataset][split]:
        if len(x["cot"]) == 0:
            c += 1
    print(c)

train
1
test
1
validation
0


In [3]:
from cot import Collection

# 1) Dataset load and selecting random sample
collection = Collection(["worldtree"], verbose=False)
collection_worldtree_10 = collection.select(split="train", number_samples=10)

# 2) Language Model generates chains of thought and then extracts answers
config={
    "debug": False,
    "instruction_keys": ['qa-01'],
    "cot_trigger_keys": ['kojima-01'],
    "answer_extraction_keys": ['kojima-A-D'],
    "warn": True,
}
collection_worldtree_10.generate(config=config)

# 3) Performance evaluation
collection_worldtree_10.evaluate()


Loading worldtree...

            You are about to  call an external API  in total 20 times, which  may produce costs .
            Number API calls for CoT generation: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1
            Number API calls for answer extraction: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1 * n_answer_extraction_keys 1
            Do you want to continue? y/n
            
{'accuracy': {'qa-01_kojima-01_kojima-A-D': 0.7}}
